# Linear Convergence of iterations viewed through Fixed-Point Iteration

In this section, we will examine an iteration on estimating the value of a function's "fixed points" in order to examine issues surrounding convergence of an iteration, depending on how we phrase the problem.

## Setting the Scene

Consider a function $g(x) = x - \tan(x)$. 

If we needed to obtain the root $g(r) = 0$, i.e., $r \in \mathbb{R}:\ 0 = r - \tan(r)$, we might run into issues with finite-precision machine arithmetic which would arise from the subtraction and division of small numbers which occur in the problem. This can occur depending on how you implement the problem since the tangent is the ratio of the sine to the cosine of an angle; although this is mititgated by the usage of $\tan(x)$ as a Taylor-approximation derived series in implementation.

So, rephrasing the root finding problem, we instead are trying to solve 

$$ r = \tan(r) $$

Where $r$ is considered a "fixed point" of the tangent function. Abstractly defined,

> **Dfn. Fixed Points** <br> A number $r \in \mathbb{R} : r = g(r)$ is a fixed point for the function $g(x)$.

If we take an initial guess for $r$, we can compute $g(r)$ and measure the difference of it from our guess, using an iteration on $g$ (which in our case, is $\tan(x)$). 

> **Dfn. Fixed Point Iteration** <br>
> Using $x_0$ as an initial guess, for $i = 1, 2, \ldots, n$ iterations, $$x_{i+1} = g(x_i)$$ 
That is, the next guess $x_i$ for each iteration is the value of our chosen function $g$ at our current guess. 

Let's look at it as a code example, and then run it on $g(x) = x - \tan(x)$ to find its roots using the iteration $x_{i+1} = \tan(x_i)$.

In [1]:
import numpy as np

def fpi(func, initial_guess, max_iter=16, abs_err=1e-05, rel_err=1e-05):
    iter_count = 1
    guess = 
    
    while iter_count <= max_iter && 
    

SyntaxError: unexpected EOF while parsing (<ipython-input-1-48862cab32d3>, line 3)

However, we'll find that iteration on a problem to find its fixed points will depend on our inital guess; which we will now examine to begin illustrating the concepts of convergence and error that form a foundation for the methods presented in this course. 

## What?

### Convergence

When we get increasingly close to an answer, we're "converging". When our operation progressively gets farther from an answer to our iteration, it "diverges".

If we let our error for each iterate be denoted as $e_i$, we can start examining what it means for us to find an answer or not (convergence of the iteration). 

We can say that our iteration is __linearly convergent__ if the the ratio of the error at each iterate to the previous error is less than 1. Notationally,

$$ \lim_{i \to \infty} \frac{e_{i+1}}{e_i} = S < 1$$

Generally, we can express convergence with a rate M using 

$$ \lim_{i \to \infty} \frac{e_{i+1}}{(e_i)^q} = M < \infty$$

where $q$ then denotes the _order_ of the convergence. For example, if $q=2$, then the convergence is _quadratic_ (the error at the current step is roughly the square at the next step, and their ratio isn't infinity. We'll see this later with the Newton-Raphson method, another iteration technique. 


### Error Tolerance

Depending on when we want to 

A measure of our precision, or absolute error, at each step is said to be 

$$ e_{i+1} = | 




## How?

## Why?

### Cobweb Diagrams

## When?

## Where?
